# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [67]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi.newsapi_client import NewsApiClient
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Alema\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [68]:
# Read your api key environment variable

api_key = os.getenv("news_api")


In [69]:
# Create a newsapi client

newsapi = NewsApiClient(api_key=api_key)


In [70]:
# Fetch the Bitcoin news articles
Bitcoin = newsapi.get_everything(
    q="Bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy")

In [71]:
# Fetch the Ethereum news articles
Ethereum = newsapi.get_everything(
    q="Ethereum",
    language="en",
    page_size=100,
    sort_by="relevancy")

In [72]:
# Create the Bitcoin sentiment scores DataFrame
Bitcoin_sentiments = []

for article in Bitcoin["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        Bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
Bitcoin_df = pd.DataFrame(Bitcoin_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
Bitcoin_df = Bitcoin_df[cols]

Bitcoin_df.head()


,date,text,compound,positive,negative,neutral
0,2021-08-06,When my wife started a little garden in our ur...,-0.3400,0.000,0.061,0.939
1,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.000,0.902
2,2021-08-10,Photo by Joe Raedle/Getty Images\r\n\n \n\n Tw...,0.4019,0.080,0.000,0.920
3,2021-08-19,Retailers are increasingly accepting cryptocur...,0.6187,0.153,0.000,0.847
4,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.000,0.839


In [73]:
# Create the Ethereum sentiment scores DataFrame
Ethereum_sentiments = []

for article in Ethereum["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        Ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
Ethereum_df = pd.DataFrame(Ethereum_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
Ethereum_df = Ethereum_df[cols]

Ethereum_df.head()


,date,text,compound,positive,negative,neutral
0,2021-07-29,Blockchain infrastructure startups are heating...,-0.2411,0.000,0.061,0.939
1,2021-08-05,Cent was founded in 2017 as an ad-free creator...,0.6956,0.190,0.000,0.810
2,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.000,0.902
3,2021-08-10,"After the bell today, Coinbase reported anothe...",0.5719,0.139,0.000,0.861
4,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.000,0.839


In [74]:
# Describe the Bitcoin Sentiment
Bitcoin_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.080781,0.055510,0.034060,0.910450
std,0.355171,0.062107,0.056722,0.077482
min,-0.893400,0.000000,0.000000,0.688000
25%,-0.038650,0.000000,0.000000,0.863500
50%,0.000000,0.052000,0.000000,0.921000
75%,0.345300,0.089500,0.066750,1.000000
max,0.883400,0.274000,0.312000,1.000000


In [75]:
# Describe the Ethereum Sentiment
Ethereum_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.00000,100.000000
mean,0.077367,0.050080,0.02831,0.921600
std,0.363142,0.058704,0.05395,0.072789
min,-0.893400,0.000000,0.00000,0.688000
25%,-0.006450,0.000000,0.00000,0.885750
50%,0.000000,0.044000,0.00000,0.928500
75%,0.323650,0.080000,0.04900,1.000000
max,0.750600,0.201000,0.31200,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum

Q: Which coin had the highest compound score?

A: Bitcoin

Q. Which coin had the highest positive score?

A: Bitcoin

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [76]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alema\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [77]:
# Instantiate the lemmatizer

lemmatizer = WordNetLemmatizer()

In [78]:


def tokenizer(text):
    lemmatizer = WordNetLemmatizer()
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    return [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in set(stopwords.words('english'))]



In [79]:
# Create a new tokens column for Bitcoin

Bitcoin_df["tokens"] = Bitcoin_df["text"].apply(tokenizer)
Bitcoin_df.head()   

,date,text,compound,positive,negative,neutral,tokens
0,2021-08-06,When my wife started a little garden in our ur...,-0.3400,0.000,0.061,0.939,"[wife, started, little, garden, urban, backyar..."
1,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.000,0.902,"[paypal, allow, user, outside, u, buy, hold, s..."
2,2021-08-10,Photo by Joe Raedle/Getty Images\r\n\n \n\n Tw...,0.4019,0.080,0.000,0.920,"[photo, joe, raedlegetty, image, twitter, squa..."
3,2021-08-19,Retailers are increasingly accepting cryptocur...,0.6187,0.153,0.000,0.847,"[retailer, increasingly, accepting, cryptocurr..."
4,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.000,0.839,"[paypal, bringing, ability, buy, hold, sell, c..."


In [80]:
def tokenizer(text):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return output

In [81]:
# Create a new tokens column for Ethereum
Ethereum_df["tokens"] = Ethereum_df["text"].apply(tokenizer)
Ethereum_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2021-07-29,Blockchain infrastructure startups are heating...,-0.2411,0.000,0.061,0.939,"[blockchain, infrastructure, startup, heating,..."
1,2021-08-05,Cent was founded in 2017 as an ad-free creator...,0.6956,0.190,0.000,0.810,"[cent, wa, founded, adfree, creator, network, ..."
2,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.000,0.902,"[paypal, allow, user, outside, us, buy, hold, ..."
3,2021-08-10,"After the bell today, Coinbase reported anothe...",0.5719,0.139,0.000,0.861,"[bell, today, coinbase, reported, another, per..."
4,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.000,0.839,"[paypal, bringing, ability, buy, hold, sell, c..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [82]:
from collections import Counter
from nltk import ngrams


In [83]:
# Generate the Bitcoin N-grams where N=2
processed = tokenizer(text)
word_counts = Counter(text)
bitcoin_counts = Counter(ngrams(processed, n=2))
bitcoin_counts

Counter({('sixth', 'time'): 1,
         ('time', 'charm'): 1,
         ('charm', 'dow'): 1,
         ('dow', 'jones'): 1,
         ('jones', 'industrial'): 1,
         ('industrial', 'average'): 1,
         ('average', 'closing'): 1,
         ('closing', 'first'): 1,
         ('first', 'time'): 1,
         ('time', 'friday'): 1,
         ('friday', 'stock'): 1,
         ('stock', 'cap'): 1,
         ('cap', 'buythedip'): 1,
         ('buythedip', 'rebound'): 1,
         ('rebound', 'selloff'): 1,
         ('selloff', 'opened'): 1,
         ('opened', 'week'): 1,
         ('week', 'see'): 1,
         ('see', 'full'): 1,
         ('full', 'story'): 1,
         ('story', 'char'): 1})

In [84]:
# Generate the Ethereum N-grams where N=2
processed = tokenizer(text)
word_counts = Counter(text)
Ethereum_counts = Counter(ngrams(processed, n=2))
Ethereum_counts

Counter({('sixth', 'time'): 1,
         ('time', 'charm'): 1,
         ('charm', 'dow'): 1,
         ('dow', 'jones'): 1,
         ('jones', 'industrial'): 1,
         ('industrial', 'average'): 1,
         ('average', 'closing'): 1,
         ('closing', 'first'): 1,
         ('first', 'time'): 1,
         ('time', 'friday'): 1,
         ('friday', 'stock'): 1,
         ('stock', 'cap'): 1,
         ('cap', 'buythedip'): 1,
         ('buythedip', 'rebound'): 1,
         ('rebound', 'selloff'): 1,
         ('selloff', 'opened'): 1,
         ('opened', 'week'): 1,
         ('week', 'see'): 1,
         ('see', 'full'): 1,
         ('full', 'story'): 1,
         ('story', 'char'): 1})

In [85]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    processed = process_text(big_string)
    bigrams = ngrams(processed, n=2)
    top_10 = dict(Counter(bigrams).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['bigram', 'count'])
   

In [86]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    return Counter(tokens).most_common(N)

In [87]:
# Use token_count to get the top 10 words for Bitcoin
token_count(bitcoin_counts)

[(('sixth', 'time'), 1),
 (('time', 'charm'), 1),
 (('charm', 'dow'), 1),
 (('dow', 'jones'), 1),
 (('jones', 'industrial'), 1),
 (('industrial', 'average'), 1),
 (('average', 'closing'), 1),
 (('closing', 'first'), 1),
 (('first', 'time'), 1),
 (('time', 'friday'), 1)]

In [88]:
# Use token_count to get the top 10 words for Ethereum
token_count(Ethereum_counts)

[(('sixth', 'time'), 1),
 (('time', 'charm'), 1),
 (('charm', 'dow'), 1),
 (('dow', 'jones'), 1),
 (('jones', 'industrial'), 1),
 (('industrial', 'average'), 1),
 (('average', 'closing'), 1),
 (('closing', 'first'), 1),
 (('first', 'time'), 1),
 (('time', 'friday'), 1)]

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [89]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [90]:
# Generate the Bitcoin word cloud
wc = WordCloud().generate(' '.join(bitcoin_counts))
plt.imshow(wc)

TypeError: sequence item 0: expected str instance, tuple found

In [91]:
# Generate the Ethereum word cloud

wc2 = WordCloud().generate(' '.join(Ethereum_counts))
plt.imshow(wc)

TypeError: sequence item 0: expected str instance, tuple found

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [92]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [93]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [95]:
# Concatenate all of the Bitcoin text together
bitcoin_text_str = " ".join(Bitcoin_df.text.to_list())
print(bitcoin_text_str)

When my wife started a little garden in our urban backyard, all I could think about were the worms. Also the bugs, and the dirt, which is of course filled with worms and bugs and composted corn cobs.… [+3499 chars] PayPal will now allow users outside the U.S. to buy, hold and sell cryptocurrency for the first time. The company announced today the launch of a new service that will allow customers in the U.K. to … [+4420 chars] Photo by Joe Raedle/Getty Images

 

 Twitter and Square CEO Jack Dorsey, a noted superfan of Bitcoin, claimed on Monday that the cryptocurrency will eventually unite the world. Yes, really.
On Mond… [+2119 chars] Retailers are increasingly accepting cryptocurrency for physical items, but did you know you can spend crypto on ephemeral experiences like travel too? As more cryptocurrency holders indicate a willi… [+2783 chars] PayPal is bringing the ability to buy, hold and sell cryptocurrencies across to the other side of the pond, the better part of a year after i

In [96]:
# Run the NER processor on all of the text
btc_doc = nlp(bitcoin_text_str)
btc_doc.user_data["Bitcoin"] = "Bitcoin NER"

# Add a title to the document
# YOUR CODE HERE!

In [97]:
# Render the visualization
displacy.render(btc_doc,style='ent')

In [98]:
# List all Entities
for ent in btc_doc.ents:
    print(ent.text, ent.label_)

U.S. GPE
first ORDINAL
today DATE
U.K. GPE
Joe Raedle PERSON
Twitter PRODUCT
Square PRODUCT
Jack Dorsey PERSON
Monday DATE
a year DATE
US GPE
Joe TidyCyber PERSON
today DATE
Coinbase PERSON
its second quarter DATE
the quarter DATE
Coinbase ORG
$2.23 billion MONEY
two years DATE
roughly 49 million CARDINAL
UK GPE
first ORDINAL
four CARDINAL
SINGAPORE GPE
July 28 DATE
Reuters ORG
40,000 MONEY
Wednesday DATE
first ORDINAL
$311 billion MONEY
Q2 2021 DATE
Insider Intellige PERSON
Motherboard PERSON
August 6, 2021 DATE
REUTERS ORG
Dado Ruvic/IllustrationAug PERSON
20 CARDINAL
Reuters ORG
5.01 % PERCENT
49,106.4 MONEY
22:04 GMT TIME
Friday DATE
August DATE
20, 2021 DATE
PennyStocksBitcoin Price Surge Helped Boost ORG
Bitcoin, Dogecoin ORG
Ethereum GPE
T-Mobile ORG
Roughly 47.8 million CARDINAL
June 7, 2021 DATE
REUTERS ORG
Edgar Su/IllustrationAug PERSON
Reuters ORG
7.07% PERCENT
47,587.38 MONEY
2200 CARDINAL
REUTERS ORG
Dado Ruvic/Illustration
Welcome PERSON
10 CARDINAL
the Opening Bell ORG


---

### Ethereum NER

In [99]:
# Concatenate all of the Ethereum text together
ethereum_text_str = " ".join(Ethereum_df.text.to_list())
print(ethereum_text_str)

Blockchain infrastructure startups are heating up as industry fervor brings more developers and users to a space that still feels extremely young despite a heavy institutional embrace of the crypto s… [+2468 chars] Cent was founded in 2017 as an ad-free creator network that allows users to offer each other crypto rewards for good posts and comments — it’s like gifting awards on Reddit, but with Ethereum. But in… [+6228 chars] PayPal will now allow users outside the U.S. to buy, hold and sell cryptocurrency for the first time. The company announced today the launch of a new service that will allow customers in the U.K. to … [+4420 chars] After the bell today, Coinbase reported another period of impressive results in its second quarter earnings report.
During the quarter, Coinbase’s total revenue reached $2.23 billion, which helped t… [+4900 chars] PayPal is bringing the ability to buy, hold and sell cryptocurrencies across to the other side of the pond, the better part of a year after i

In [101]:
# Run the NER processor on all of the text
eth_doc = nlp(ethereum_text_str)
eth_doc.user_data["Ethereum"] = "Ethereum NER"

# Add a title to the document
# YOUR CODE HERE!

In [102]:
# Render the visualization
displacy.render(eth_doc,style='ent')

In [ ]:
# List all Entities
for ent in eth_doc.ents:
    print(ent.text, ent.label_)

---